In [1]:
from pyexpat import model
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import glob
import cv2
import os
import locale

import tensorflow as tf
#import all necessary layers
from tensorflow.keras.layers import Input, DepthwiseConv2D
from tensorflow.keras.layers import Conv2D, BatchNormalization
from tensorflow.keras.layers import ReLU, AvgPool2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.losses import BinaryCrossentropy

from sklearn.model_selection import train_test_split
import matplotlib.image as mpimg
from tensorflow.keras.layers import concatenate
from deepforest import CascadeForestClassifier
from tensorflow.keras.optimizers import Adam

In [2]:
# MobileNet block
def mobilnet_block (x, filters, strides):
    
    x = DepthwiseConv2D(kernel_size = 3, strides = strides, padding = 'same')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    x = Conv2D(filters = filters, kernel_size = 1, strides = 1)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    
    return x

In [3]:
def create_cnn_mobilenet ():
     input = Input(shape = (320,320,9))
     x = Conv2D(filters = 32, kernel_size = 3, strides = 2, padding = 'same')(input)
     x = BatchNormalization()(x)
     x = ReLU()(x)

     x = mobilnet_block(x, filters = 64, strides = 1)
     x = mobilnet_block(x, filters = 128, strides = 2)
     x = mobilnet_block(x, filters = 128, strides = 1)
     x = mobilnet_block(x, filters = 256, strides = 2)
     x = mobilnet_block(x, filters = 256, strides = 1)
     x = mobilnet_block(x, filters = 512, strides = 2)
     for _ in range (5):
          x = mobilnet_block(x, filters = 512, strides = 1)
     x = mobilnet_block(x, filters = 1024, strides = 2)
     x = mobilnet_block(x, filters = 1024, strides = 1)
     x = AvgPool2D (pool_size = 7, strides = 1, data_format='channels_first')(x)
     output = Flatten()(x)
     output = Dense(1)(output)
     model = Model(inputs=input, outputs=output)
     return model

In [4]:
gasdata = pd.read_csv("/Users/Eddie/Downloads/data5 (1).csv")
data_dir = '/Users/Eddie/Downloads/Data5'
imagedata = sorted(os.listdir(data_dir))
print(len(imagedata))
X_data = []
for image in imagedata:
        # print(image)
        img = mpimg.imread('/Users/Eddie/Downloads/Data5/'+image)
        img = img.reshape(320,320,9)
        img = img/255.0
        X_data.append(img)
images = np.array(X_data)
print(images.shape)


1000
(1000, 320, 320, 9)


In [5]:
split = train_test_split(gasdata, images, test_size=0.2)
(trainAttrX, testAttrX, trainImagesX, testImagesX) = split
trainy = trainAttrX["Gas"]
testy = testAttrX["Gas"]
trainAttrX = trainAttrX.drop(columns=['Gas'])
testAttrX = testAttrX.drop(columns = ['Gas'])
trainAttrX= (trainAttrX - np.min(trainAttrX)) / (np.max(trainAttrX) - np.min(trainAttrX))
testAttrX = (testAttrX - np.min(testAttrX)) / (np.max(testAttrX) - np.min(testAttrX))
print(trainAttrX.shape)
print(testAttrX.shape)
print(trainy.shape)
print(testy.shape)

(800, 2)
(200, 2)
(800,)
(200,)


In [6]:
# print(testy)
newtesty = []
for value in testy:
    value = float(value)
    newtesty.append(value)
# print(newtesty)
newtrainy = []
for value in trainy:
    value = float(value)
    newtrainy.append(value)

print(newtrainy)

[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 1.0,

In [7]:
cnn = create_cnn_mobilenet()
X = cnn.predict(trainImagesX)
print(X.shape)





25/25 [==============================] - 13s 481ms/step
(800, 1)


In [8]:
X = np.array(X)
print(X)
trainAttrX = np.array(trainAttrX)
trainAttrX.resize(800,2)
newtrainy=np.array(newtrainy)
newtrainy = newtrainy.reshape(-1,1)
print()
print(newtrainy)
print(newtesty)
print(trainAttrX.shape)
print(trainAttrX)
traindata = concatenate([X,trainAttrX])
traindata = np.array(traindata)
traindata.reshape(-1,1)
print(traindata)
print(traindata.shape)



[[4.9730568e-23]
 [1.8289409e-23]
 [7.6298841e-23]
 [5.9982143e-23]
 [5.0278694e-23]
 [7.8884494e-23]
 [8.5689284e-23]
 [6.4294019e-23]
 [2.4151961e-23]
 [7.2705043e-23]
 [3.8327735e-23]
 [4.9606868e-23]
 [7.1296012e-23]
 [5.6351364e-23]
 [5.4795667e-23]
 [7.5305015e-23]
 [5.2885980e-23]
 [2.6043533e-23]
 [7.4401151e-23]
 [7.8183588e-23]
 [7.1605895e-23]
 [7.8383498e-23]
 [5.0359495e-23]
 [5.6246338e-23]
 [7.1563196e-23]
 [2.2851603e-23]
 [6.4694357e-23]
 [6.7069560e-23]
 [4.3237760e-23]
 [6.0712464e-23]
 [7.7347648e-23]
 [6.0376485e-23]
 [7.7881549e-23]
 [9.3594629e-23]
 [7.2734780e-23]
 [7.5579690e-23]
 [6.8863310e-23]
 [7.3986147e-23]
 [5.0560765e-23]
 [6.8973460e-23]
 [7.5976835e-23]
 [4.7503333e-23]
 [3.0824891e-23]
 [5.9418461e-23]
 [5.5298695e-23]
 [7.6324552e-23]
 [4.0597332e-23]
 [8.8377078e-23]
 [2.5592907e-23]
 [2.7038602e-23]
 [7.0603587e-23]
 [7.7432119e-23]
 [4.4734898e-23]
 [6.3550369e-23]
 [4.0034909e-23]
 [7.1703278e-23]
 [2.6927079e-23]
 [9.4949899e-23]
 [3.4505054e-2

In [9]:
model = CascadeForestClassifier(n_estimators=50,random_state=1)
hist = model.fit(traindata,trainy)

[2023-12-29 20:54:26.444] Start to fit the model:
[2023-12-29 20:54:26.444] Fitting cascade layer = 0 
[2023-12-29 20:54:34.807] layer = 0  | Val Acc = 98.500 % | Elapsed = 8.363 s
[2023-12-29 20:54:34.824] Fitting cascade layer = 1 
[2023-12-29 20:54:45.155] layer = 1  | Val Acc = 99.000 % | Elapsed = 10.331 s
[2023-12-29 20:54:45.165] Fitting cascade layer = 2 
[2023-12-29 20:54:54.065] layer = 2  | Val Acc = 98.500 % | Elapsed = 8.900 s
[2023-12-29 20:54:54.065] Early stopping counter: 1 out of 2
[2023-12-29 20:54:54.071] Fitting cascade layer = 3 
[2023-12-29 20:55:03.088] layer = 3  | Val Acc = 98.375 % | Elapsed = 9.016 s
[2023-12-29 20:55:03.088] Early stopping counter: 2 out of 2
[2023-12-29 20:55:03.088] Handling early stopping
[2023-12-29 20:55:03.095] The optimal number of layers: 2


In [22]:
gasdata = pd.read_csv("/Users/eddie/Downloads/Gas Sensors Data - Sheet1 (3) (1).csv")
result = gasdata['Gas']
gasdata = gasdata.drop(columns=['Gas'])
gasdata= (gasdata - np.min(gasdata)) / (np.max(gasdata) - np.min(gasdata))
images =  os.listdir("/Users/eddie/Downloads/No Gas 4/No Gas")
X_dat = []
for image in images:
       print(image)
       img = mpimg.imread('/Users/eddie/Downloads/No Gas 4/No Gas/'+image)
       img.resize(320,320,9)
       img = img/255.0
       X_dat.append(img)
test_images = np.array(X_dat)
print(test_images.shape)

001.PNG
002.PNG
003.PNG
004.PNG
005.PNG
006.PNG
007.PNG
008.PNG
009.PNG
010.PNG
011.PNG
012.PNG
013.PNG
014.PNG
015.PNG
016.PNG
017.PNG
018.PNG
019.PNG
020.PNG
021.PNG
022.PNG
023.PNG
024.PNG
025.PNG
026.PNG
027.PNG
028.PNG
029.PNG
030.PNG
031.PNG
032.PNG
033.PNG
034.PNG
035.PNG
036.PNG
037.PNG
038.PNG
039.PNG
040.PNG
041.PNG
042.PNG
043.PNG
044.PNG
045.PNG
046.PNG
047.PNG
048.PNG
049.PNG
050.PNG
051.PNG
052.PNG
053.PNG
054.PNG
055.PNG
056.PNG
057.PNG
058.PNG
059.PNG
060.PNG
061.PNG
062.PNG
063.PNG
064.PNG
065.PNG
066.PNG
067.PNG
068.PNG
069.PNG
070.PNG
071.PNG
072.PNG
073.PNG
074.PNG
075.PNG
076.PNG
077.PNG
078.PNG
079.PNG
080.PNG
081.PNG
082.PNG
083.PNG
084.PNG
085.PNG
086.PNG
087.PNG
088.PNG
089.PNG
090.PNG
091.PNG
092.PNG
093.PNG
094.PNG
095.PNG
096.PNG
097.PNG
098.PNG
099.PNG
100.PNG
101.PNG
102.PNG
103.PNG
104.PNG
105.PNG
106.PNG
107.PNG
108.PNG
109.PNG
110.PNG
111.PNG
112.PNG
113.PNG
114.PNG
115.PNG
116.PNG
117.PNG
118.PNG
119.PNG
120.PNG
121.PNG
122.PNG
123.PNG
124.PNG
125.PNG


In [23]:
#print(test_images.shape)
cows = cnn.predict(test_images)

6/6 [==============================] - 3s 431ms/step


In [24]:
cows = np.array(cows)
# print(cows)
print(cows.shape)
print(testAttrX.shape)
# testAttrX.resize(122,2)
# print(gasdata)
# print(gasdata.shape)
test = concatenate([cows,gasdata])


(176, 1)
(200, 2)


In [25]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [26]:
performance = model.predict(test)
print(performance)
cows = np.array(testy)
performance.round()
actual = []
for value in performance: 
    print(value)
    if(value>=0.5):
            actual.append(1)
    else:
            actual.append(0)

acutal = np.array(actual)
print(actual)
testy = result
testy = np.array(testy)
with np.printoptions(threshold=np.inf):
        print(testy)
accuracy = accuracy_score(testy, actual)
print('Accuracy: %f' % accuracy)
precision = precision_score(testy, actual)
print('Precision: %f' % precision)
recall = recall_score(testy, actual)
print('Recall: %f' % recall)
f1 = f1_score(testy, actual)
print('F1 score: %f' % f1)

[2023-12-29 21:07:46.223] Start to evalute the model:
[2023-12-29 21:07:46.223] Evaluating cascade layer = 0 
[2023-12-29 21:07:46.457] Evaluating cascade layer = 1 
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 